In [ ]:
#dataset
#https://www.kaggle.com/datasets/uom190346a/sleep-health-and-lifestyle-dataset


In [ ]:
#path

# "/content/drive/My Drive/ML_Project/Sleep_health_and_lifestyle_dataset.csv"


In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler # Import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from scipy.stats import mode
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score, recall_score, classification_report

In [ ]:
# Google Drive link
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Load the dataset
path = '/content/drive/My Drive/ML_Project/Sleep_health_and_lifestyle_dataset.csv'
initial_data = pd.read_csv(path)

In [ ]:
initial_data.head()

In [ ]:
# Drop unnecessary columns(Feature selection)
mydata = initial_data.drop(['Person ID', 'Occupation', 'Sleep Disorder'], axis=1, errors='ignore')


In [ ]:
# Display the cleaned data (Feature display)
print("Cleaned Data:")
mydata.head()

In [ ]:
# Encode categorical variables
X = mydata.drop('Stress Level', axis=1)
y = mydata['Stress Level']
X = pd.get_dummies(X, drop_first=True)

In [ ]:
X.head()

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# SVM
svm_classifier = SVC(probability=True, kernel='rbf', random_state=42)
svm_classifier.fit(X_train, y_train)
svm_pred = svm_classifier.predict(X_test)
svm_accuracy = accuracy_score(y_test, svm_pred)

In [ ]:
# Create a LabelEncoder object
label_encoder = LabelEncoder()

# Fit the encoder to your target variable and transform it
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

# XGBoost Classifier
xgb_classifier = XGBClassifier(n_estimators=100, learning_rate=0.1, random_state=42, use_label_encoder=False, eval_metric='logloss')
xgb_classifier.fit(X_train, y_train)
xgb_pred = xgb_classifier.predict(X_test)
xgb_accuracy = accuracy_score(y_test, xgb_pred)

In [ ]:
# Logistic Regression
logreg_classifier = LogisticRegression(max_iter=1000, random_state=42)
logreg_classifier.fit(X_train, y_train)
logreg_pred = logreg_classifier.predict(X_test)
logreg_accuracy = accuracy_score(y_test, logreg_pred)

In [ ]:
# Define recommendation and action plan functions
def content_based_recommendation(stress_level, physical_activity, sleep_quality, bmi, blood_pressure):
    if stress_level > 7:
        return "Meditation, Yoga, Reduce screen time, Regular Sleep"
    elif physical_activity < 5:
        return "Increase Physical Activity, Morning Walk, Breathing Exercises"
    elif sleep_quality < 5:
        return "Improve Sleep Hygiene, Avoid Caffeine, Create Sleep Schedule"
    elif bmi > 30:
        return "Consult a nutritionist, Begin a weight loss program"
    elif blood_pressure > 140:
        return "Consult with a healthcare provider for blood pressure management"
    else:
        return "Maintain Current Lifestyle, Stay Positive"

In [ ]:
#performance of XGBoost
print("\nXGBoost Results:")
print(f"Accuracy: {xgb_accuracy:.2f}")
print(classification_report(y_test, xgb_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, xgb_pred))

In [ ]:
# Create a StandardScaler instance and fit it to your training data
scaler = StandardScaler()  # Create a StandardScaler object
scaler.fit(X_train)  # Fit the scaler to your training data

In [ ]:
# Function to test a single person's data
def test_single_person(gender, age, sleep_duration, physical_activity, bmi, blood_pressure, heart_rate, daily_steps):

    # Single input data (Modified to match original dataset structure)
    person_data = pd.DataFrame({
        'Gender': [gender],
        'Age': [age],
        'Sleep Duration': [sleep_duration],
        'Physical Activity Level': [physical_activity],
        'BMI': [bmi],
        'Blood Pressure': [blood_pressure],
        'Heart Rate': [heart_rate],
        'Daily Steps': [daily_steps]
    }, index=[0])  # Adding an index to avoid potential issues

    # Preprocess person_data to match the format used during training:
    # 1. One-hot encode categorical features
    person_data = pd.get_dummies(person_data, columns=['Gender'], drop_first=True)

    # 2. Ensure all columns from training data are present
    #    (Add missing columns with 0 values)
    for col in X_train.columns:  # X_train has the expected columns
        if col not in person_data.columns:
            person_data[col] = 0

    # 3. Reorder columns to match the order in training data
    person_data = person_data[X_train.columns]

    # Now you can safely scale the data:
    person_scaled = scaler.transform(person_data)

    # Predict stress level using the models
    svm_pred = svm_classifier.predict(person_data)[0]
    xgb_pred = xgb_classifier.predict(person_data)[0]
    logreg_pred = logreg_classifier.predict(person_data)[0]
    # Combine predictions using voting (majority voting for categorical output)
    predictions = [svm_pred, xgb_pred, logreg_pred]
    final_pred = max(set(predictions), key=predictions.count)  # Majority voting

    # Get stress relief recommendations
    recommendation = content_based_recommendation(
        stress_level=final_pred,  # Using the combined prediction
        physical_activity=physical_activity,
        sleep_quality=sleep_duration,
        bmi=bmi,
        blood_pressure=blood_pressure
    )


    # Output the results
    print(f"\nTest for Single Person's Data:")
    print(f"Gender: {gender}")
    print(f"Age: {age}")
    print(f"Sleep Duration: {sleep_duration} hours")
    print(f"Physical Activity Level: {physical_activity}")
    print(f"BMI: {bmi}")
    print(f"Blood Pressure: {blood_pressure} mmHg")
    print(f"Heart Rate: {heart_rate} bpm")
    print(f"Daily Steps: {daily_steps}")
    print(f"Predicted Stress Level (SVM): {svm_pred}")
    print(f"Predicted Stress Level (XGBoost): {xgb_pred}")
    print(f"Predicted Stress Level (Logistic Regression): {logreg_pred}")
    test_and_predict_stress_level_with_meta_model(gender, age, sleep_duration, physical_activity, bmi, blood_pressure, heart_rate, daily_steps)


In [ ]:
def get_base_model_predictions(X_train):
    svm_probs = svm_classifier.predict_proba(X_train)
    xgb_probs = xgb_classifier.predict_proba(X_train)
    # Combine probabilities for the meta-model
    meta_features = np.hstack((svm_probs, xgb_probs))
    return meta_features


In [ ]:
def test_and_predict_stress_level_with_meta_model(
    gender, age, sleep_duration, physical_activity, bmi, blood_pressure, heart_rate, daily_steps
):

    # Single input data (Modified to match original dataset structure)
    person_data = pd.DataFrame({
        'Gender': [gender],
        'Age': [age],
        'Sleep Duration': [sleep_duration],
        'Physical Activity Level': [physical_activity],
        'BMI': [bmi],
        'Blood Pressure': [blood_pressure],
        'Heart Rate': [heart_rate],
        'Daily Steps': [daily_steps]
    })

    # Preprocess the input to match training data
    person_data = pd.get_dummies(person_data, columns=['Gender'], drop_first=True)

    # Add missing columns with 0 values
    for col in X_train.columns:
        if col not in person_data.columns:
            person_data[col] = 0

    # Reorder columns to match the training data
    person_data = person_data[X_train.columns]

    # Scale the input data
    person_scaled = scaler.transform(person_data)

    # Get predictions from base models for the single person's data
    svm_probs_person = svm_classifier.predict_proba(person_scaled)
    xgb_probs_person = xgb_classifier.predict_proba(person_scaled)

    # Combine probabilities for the meta-model input for the single person
    meta_features_person = np.hstack((svm_probs_person, xgb_probs_person))

    # Train the meta-model

    meta_model = LogisticRegression(max_iter=1000, random_state=42)
    meta_model.fit(get_base_model_predictions(X_train), y_train)


    # Meta-model prediction using the single person's meta features
    meta_pred = meta_model.predict(meta_features_person)


    # Recommendations based on stress level
    recommendation = content_based_recommendation(
        stress_level=meta_pred[0],  # Use combined prediction
        physical_activity=physical_activity,
        sleep_quality=sleep_duration,
        bmi=bmi,
        blood_pressure=blood_pressure
    )

    # Output the results
    print(f"\nMeta Model:")
    print(f"Predicted Stress Level (Meta-Model): {meta_pred[0]}")
    print(f"Recommendations: {recommendation}")

In [ ]:
# Calculate F1 scores
svm_f1_score = f1_score(y_test, svm_pred, average='weighted')
xgb_f1_score = f1_score(y_test, xgb_pred, average='weighted')
logreg_f1_score = f1_score(y_test, logreg_pred, average='weighted')

# Get meta_pred for F1 score calculation
# Assuming you want to use the same test data (X_test) for meta-model prediction
meta_features_test = get_base_model_predictions(X_test)
meta_model = LogisticRegression(max_iter=1000, random_state=42)
meta_model.fit(get_base_model_predictions(X_train), y_train)  # Fit meta-model
meta_pred = meta_model.predict(meta_features_test)  # Predict using test data

meta_model_f1_score = f1_score(y_test, meta_pred, average='weighted')
meta_model_accuracy = accuracy_score(y_test, meta_pred)


# Display F1 scores
print("\nF1 Scores:")
print(f"SVM F1 Score: {svm_f1_score:.2f}")
print(f"XGBoost F1 Score: {xgb_f1_score:.2f}")
print(f"Logistic Regression F1 Score: {logreg_f1_score:.2f}")
print(f"Meta Model F1 Score: {meta_model_f1_score:.2f}")


In [ ]:
# Display all model accuracies
print("\nModel Accuracies:")
print(f"SVM Accuracy: {svm_accuracy:.2f}")
print(f"XGBoost Accuracy: {xgb_accuracy:.2f}")
print(f"Logistic Regression Accuracy: {logreg_accuracy:.2f}")
print(f"Meta Model Accuracy: {meta_model_accuracy:.2f}")

In [ ]:
# Calculate Precision and Recall for each model
svm_precision = precision_score(y_test, svm_pred, average='weighted')
svm_recall = recall_score(y_test, svm_pred, average='weighted')

xgb_precision = precision_score(y_test, xgb_pred, average='weighted')
xgb_recall = recall_score(y_test, xgb_pred, average='weighted')

logreg_precision = precision_score(y_test, logreg_pred, average='weighted')
logreg_recall = recall_score(y_test, logreg_pred, average='weighted')

# For the Meta Model, use the meta_pred calculated previously on the test set
meta_model_precision = precision_score(y_test, meta_pred, average='weighted')
meta_model_recall = recall_score(y_test, meta_pred, average='weighted')

# Display Precision and Recall
print("\nPrecision Scores:")
print(f"SVM Precision: {svm_precision:.2f}")
print(f"XGBoost Precision: {xgb_precision:.2f}")
print(f"Logistic Regression Precision: {logreg_precision:.2f}")
print(f"Meta Model Precision: {meta_model_precision:.2f}")

print("\nRecall Scores:")
print(f"SVM Recall: {svm_recall:.2f}")
print(f"XGBoost Recall: {xgb_recall:.2f}")
print(f"Logistic Regression Recall: {logreg_recall:.2f}")
print(f"Meta Model Recall: {meta_model_recall:.2f}")

# Alternatively, you can print the classification report for a more detailed view:
print("\nClassification Reports:")

print("\nSVM Classification Report:")
print(classification_report(y_test, svm_pred))

print("\nXGBoost Classification Report:")
print(classification_report(y_test, xgb_pred))

print("\nLogistic Regression Classification Report:")
print(classification_report(y_test, logreg_pred))

print("\nMeta Model Classification Report:")
print(classification_report(y_test, meta_pred))

In [ ]:
# Display Confusion Matrices
print("\nConfusion Matrices:")

print("\nSVM Confusion Matrix:")
print(confusion_matrix(y_test, svm_pred))

print("\nXGBoost Confusion Matrix:")
print(confusion_matrix(y_test, xgb_pred))

print("\nLogistic Regression Confusion Matrix:")
print(confusion_matrix(y_test, logreg_pred))

print("\nMeta Model Confusion Matrix:")
print(confusion_matrix(y_test, meta_pred))

In [ ]:
test_single_person(
    gender='Male',             # Gender: 'Male' or 'Female'
    age=24,                    # Age in years
    sleep_duration=1,          # Sleep Duration in hours
    physical_activity=2,       # Physical Activity level (1-10)
    bmi=25,                    # Body Mass Index
    blood_pressure=100,        # Blood Pressure in mmHg (e.g., 120/80)
    heart_rate=100,             # Heart Rate in beats per minute
    daily_steps=20000           # Daily Steps count
)